<center> <img src='https://thecreditpros.com/wp-content/uploads/2017/05/Credit-Offers-1.jpg'> </center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1.Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv(r'/kaggle/input/credit-card-customers/BankChurners.csv')

In [ ]:
df.head(10)

In [ ]:
pd.set_option('display.max_columns', None)#to view all the columns
df.head()

# 2. Exploratory Data Analysis

In [ ]:
#renaming some columns 
df = df.rename(columns={'CLIENTNUM': 'phone', 'Customer_Age': 'age', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1': 'naive_bayes_month_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2': 'naive_bayes_month_2'})

In [ ]:
df.head()

In [ ]:
df.describe()#descriptive summary

credit limit, avg oepn to buy, total_trans_amt seems to have outliers

In [ ]:
df.isnull().sum()#checking for null or missing values

there are no null/missing values SIGH!

In [ ]:
#attrition flag is our target variable
df["Attrition_Flag"].value_counts()

In [ ]:
colors=('#453CC9', '#22AFF1')
explode=[0,0.1]
df["Attrition_Flag"].value_counts().plot(kind='pie',shadow=True,explode=explode,colors=colors,autopct='%.2f',figsize=(8,6))
plt.title('Ratio of class')
plt.show()

We can see that there are very few rows for 'attrited customer' we might need to oversample the data

In [ ]:
colors=('#165081', '#29ABF4')
explode=[0,0.1]
df["Gender"].value_counts().plot(kind='pie',shadow=True,explode=explode,colors=colors,autopct='%.2f',figsize=(8,6))
plt.title('Ratio of Gender')
plt.show()

We can see that there are more number of female who uses the card than males

In [ ]:
df['Education_Level'].value_counts().plot(kind='bar',colormap='winter',figsize=(15,5))
plt.title('Frequency of Education')

We can see that graduate holds the maximum no of credit cards followed by high school 

In [ ]:
colors=('#031B88', '#6096FD', '#AAB6FB','#FB7B8E')
explode=[0.02,0.04,0.06,0.08]
df["Marital_Status"].value_counts().plot(kind='pie',shadow=True,explode=explode,colors=colors,autopct='%.2f',figsize=(8,6))
plt.title('Ratio of Maritial Status')
plt.show()

Married people use the credit card more than single people followed by unknown.

In [ ]:
colors=('#192e5b', '#1d65a6', '#72a2c0','#00743f','#FB7B8E','#AAB6FB')
explode=[0.1,0.08,0.06,0.05,0.04,0]
df["Income_Category"].value_counts().plot(kind='pie',shadow=True,explode=explode,colors=colors,autopct='%.2f',figsize=(8,6))
plt.title('Ratio of Income  Category')
plt.show()

We can see that the people with less income are the ones who uses credit card the most

In [ ]:
df.head()

In [ ]:
df['Card_Category'].value_counts()

In [ ]:
colors=('#0000CD', '#C0C0C0', '#FFD700','#e5e4e2')
explode=[0,0.2,0.2,0.3]
df["Card_Category"].value_counts().plot(kind='pie',shadow=True,explode=explode,colors=colors,autopct='%.2f',figsize=(8,6))
plt.title('Ratio of Card Category')
plt.show()

Blue cardholders are maximum.

In [ ]:
df['Income_Category'].value_counts()

In [ ]:
#encoding categorical columns
df['Attrition_Flag']=df['Attrition_Flag'].replace({'Existing Customer':0, 'Attrited Customer':1})
df['Gender']=df['Gender'].replace({'M':1,'F':0})
df['Education_Level']=df['Education_Level'].replace({'Uneducated':0, 'High School':1, 'College':2, 'Graduate':3, 'Post-Graduate':4, 'Doctorate':5, 'Unknown':6})
df['Marital_Status']=df['Marital_Status'].replace({'Single':0, 'Married':1, 'Divorced':2, 'Unknown':3})
df['Income_Category']=df['Income_Category'].replace({'Less than $40K':0, '$40K - $60K':1, '$60K - $80K':2, '$80K - $120K':3, '$120K +':4, 'Unknown':5})
df['Card_Category']=df['Card_Category'].replace({'Blue': 0, 'Silver':1,'Gold':2,'Platinum':3})

In [ ]:
df.dtypes#verifying dtypes

In [ ]:
plt.subplots(figsize=(15,8))
plt.title('Correlation between variables')
sns.heatmap(df.corr(),annot=True,cmap='YlGnBu')

In [ ]:
df.hist(bins=50, figsize=(20,20))

In [ ]:
#dropping unncecessary columns
df.drop(['naive_bayes_month_1','naive_bayes_month_2'], axis =1, inplace=True)

In [ ]:
df.shape

# 3. Splitting data into train and test,scaling and oversampling

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df.columns

In [ ]:
X=df[['phone', 'age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']]
y=df['Attrition_Flag']

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
# 80% training set 20% percent test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,shuffle=True)

In [ ]:
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
print('Shape of Training data: ',X_train.shape, y_train.shape,
     'Shape of test data: ',X_test.shape, y_test.shape)

# 4. Model Building

## 4.1 Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix, accuracy_score

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
lr_pred=lr.predict(X_test)

In [ ]:
print('F1-Score: ',f1_score(y_test,lr_pred))

In [ ]:
print('Report: ',classification_report(y_test,lr_pred))

In [ ]:
confusion_matrix(y_test,lr_pred)

In [ ]:
accuracy_score(y_test,lr_pred)

## 4.2 Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#after pruning
dt=DecisionTreeClassifier(criterion='entropy',max_depth=11)
dt.fit(X_train,y_train)

In [ ]:
dt_pred=dt.predict(X_test)

In [ ]:
print('Report: ',classification_report(y_test,dt_pred))

In [ ]:
print('Accuracy: ',accuracy_score(y_test,dt_pred))


In [ ]:
confusion_matrix(y_test,dt_pred)

### 4.2.1 Decision Tree Pruning

In [ ]:
#decision tree pruning
max_depth=[]
acc_gini=[]
acc_entropy=[]
for i in range(1,50,5):
    dtree=DecisionTreeClassifier(criterion='gini',max_depth=i)
    dtree.fit(X_train,y_train)
    pred=dtree.predict(X_test)
    
    acc_gini.append(accuracy_score(y_test,pred))
    
    dtree=DecisionTreeClassifier(criterion='entropy',max_depth=i)
    dtree.fit(X_train,y_train)
    pred=dtree.predict(X_test)
    
    acc_entropy.append(accuracy_score(y_test,pred))
    
    max_depth.append(i)
    
    d=pd.DataFrame({'acc_gini': pd.Series(acc_gini),
                   'acc_entropy': pd.Series(acc_entropy),
                   'max_depth': pd.Series(max_depth)})
    
plt.plot('max_depth','acc_gini', data=d, label='gini')
plt.plot('max_depth','acc_entropy', data=d, label='entropy')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.grid()
plt.legend()

### criterion= 'entropy' and max _depth=11 gives the accuracy upto 95%


<table >
  <tr>
    <th>Accuracy before pruning</th>
    <th>Accuracy after pruning</th> 
  
  </tr>
  <tr>
    <td>93 %</td>
    <td>94 %</td>
  </tr>
  
</table>

## 4.3 Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
rf_pred=rf.predict(X_test)

In [ ]:
print('Report: ',classification_report(y_test,rf_pred))

In [ ]:
confusion_matrix(y_test,rf_pred)

# 5. Result Comparison

<table >
  <tr>
    <th>Algorithm</th>
    <th>Accuracy</th> 
  
  </tr>
  <tr>
    <td>Logistic Regression</td>
    <td>88.6 %</td>
  </tr>
    <tr>
    <td>Decision Tree</td>
    <td>94 %</td>
  </tr>
    <tr>
    <td>Random Forest </td>
    <td>97 %</td>
  </tr>
  
</table>
<br>
<br>
<center> Out of all Random Forest outperformed other algorithms with accuracy of 97%<center>

# <center> Thank you! </center>